# **Primeiro passo**

Primeiramente iremos utilizar um código já contruido do pydantic apenas para entender o funcionamento básico do mesmo

Instalando depêndencias e importando bibliotecas necessárias

In [ ]:
!pip install pydantic
!pip install pydantic[email]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 8.7 MB/s eta 0:00:00


In [ ]:
from enum import auto, IntFlag
from typing import Any
from pydantic import (BaseModel, EmailStr, Field, SecretStr, ValidationError, )

Agora criamos classes, no caso temos:

*  'Role': que será as funções que o indíduo podera desenpenhar, (autor, editor, desenvolvedor e o admin que possue todas as anteriores).

*   'User': utilizando o 'BaseModel', que em resumo permite implementar a os basicos da nossa validação, temos o nome, email, senha e 'Role' que ja foi mencionada.

Um ponto que temos a detacar é função 'Fild' que pode ser usada para expessificar aquela variável, como: dar exemplos, dar descrições, 'Frozen = True' que seria uma ferramenta que impedisse a alteração, como por exemplo impedir que o email cadastrado seja alterado.

In [ ]:
class Role(IntFlag):
  Author = auto()
  Editor = auto()
  Developer = auto()
  Admin = Author | Editor | Developer

class User(BaseModel):
    name: str = Field(examples=["Arjan"]) #'str' garante que o campo seja um string, e como citado anteriormente temos um exemplo dentro de 'Fild'.
    email: EmailStr = Field( #'Emailstr' faz um trabalho parecido com str, nesse caso garante que esse campo seja um email.
        examples=["example@arjancodes.com"], #exemplo.
        description="The email address of the user", #descrição.
        frozen=True, #indica que esse campo não pode ser alterado posteriormente.
    )
    password: SecretStr = Field(#'Secretstr' garante que seja uma senha, além de mostrar ela assim: '*********'.
        examples=["Password123"], description="The password of the user"
    )
    role: Role = Field(default=None, description="The role of the user") #default indica que, a priori a sua possição/papel será 0, ou a mais básica possível

Temos um função que valida se os dados enseridos dentro da classe selecionada, estão de acordo com a estrutura.

In [ ]:
def validate(data: dict[str, Any]) -> None:
    try:
        user = User.model_validate(data)
        print(user) #Caso estiver certo iremos printar o usuário.
    except ValidationError as e:
        print("User is invalid") #Caso não estiver certo, iremos printar que o usuário está errado e o erro cometido.
        for error in e.errors():
            print(error)

Agora criamos um função que servira como exemplo, possuindo 'good_data' e 'bad_data' para que a nossa validação possa trabalhar.

In [ ]:
def main() -> None:
    good_data = {
        "name": "Arjan",
        "email": "example@arjancodes.com",
        "password": "Password123",
    }
    bad_data = {"email": "", "password": ""}

    validate(good_data) #exemplo de dados bons
    validate(bad_data) #exemplos de dados ruin

Uma visualização do que fizemos até agora, explicando e apontados os erros, além de gerar a biblioteca para esclarecer duvidas.

In [ ]:
if __name__ == "__main__":
  main()

name='Arjan' email='example@arjancodes.com' password=SecretStr('**********') role=None
User is invalid
{'type': 'missing', 'loc': ('name',), 'msg': 'Field required', 'input': {'email': '', 'password': ''}, 'url': 'https://errors.pydantic.dev/2.10/v/missing'}
{'type': 'value_error', 'loc': ('email',), 'msg': 'value is not a valid email address: An email address must have an @-sign.', 'input': '', 'ctx': {'reason': 'An email address must have an @-sign.'}}


# **Segundo Passo**

Acabamos de criar uma validação, porém ela é uma básica que já foi feita pelo Pydantic, logo podemos explorar essa biblioteca para fazer o nosso próprio código de validação pesonalizada

In [ ]:
import enum
import hashlib
import re
from typing import Any
from pydantic import (BaseModel, EmailStr, field_validator, model_validator, Field, SecretStr, ValidationError, )

Usando o regex para implementar regras para validação da senha e nome de usuário

In [ ]:
VALID_PASSWORD_REGEX = re.compile(r"^(?=.*[a-z])(?=.*[A-Z])(?=.*\d).{8,}$") #Indica que deve haver pelomenos 1 letra maiuscula, 1 minuscula, 1 número e 8 caracteres
VALID_NAME_REGEX = re.compile(r"^[a-zA-Z]{2,}$") #Só aceita letras maiusculas e minusculas, e deve haver no mínimo 2 caracteres

Algumas alteração em relação a prática anterior:

*   class role agora recebe numeros inteiros para relacionar às devidas funções

*   Dentro do user, adicionamos agora o '@fild_validator', basicamento iremos fazer uma validação personalizada, no caso desse exemplo, utilizaremos o 'VALID_PASSWORD_REGEX' e 'VALID_NAME_REGEX' que criamos anteriormente, além de regras próprias para a role, isso indica que essas regras podem ser expressas antes ou depois, como o nome e senha foram feitos antes e a role foi definida durante o codigo.




In [ ]:
class Role(enum.IntFlag):
    Author = 1
    Editor = 2
    Admin = 4
    SuperAdmin = 8

In [ ]:
class User(BaseModel):
    name: str = Field(examples=["Arjan"])
    email: EmailStr = Field(
        examples=["user@arjancodes.com"],
        description="The email address of the user",
        frozen=True,
    )
    password: SecretStr = Field(
        examples=["Password123"], description="The password of the user"
    )
    role: Role = Field(
        default=None, description="The role of the user", examples=[1, 2, 4, 8]
    )

    @field_validator("name")
    @classmethod
    def validate_name(cls, v: str) -> str:
        if not VALID_NAME_REGEX.match(v):
            raise ValueError(
                "Name is invalid, must contain only letters and be at least 2 characters long"
            )
        return v

    @field_validator("role", mode="before")
    @classmethod
    def validate_role(cls, v: int | str | Role) -> Role:
        op = {int: lambda x: Role(x), str: lambda x: Role[x], Role: lambda x: x}
        try:
            return op[type(v)](v)
        except (KeyError, ValueError):
            raise ValueError(
                f"Role is invalid, please use one of the following: {', '.join([x.name for x in Role])}"
            )

    @model_validator(mode="before") #Podemos elevar essa validação, criando varios pré-requisitos, aqui garantiremos que haja nome e senha, que o nome não esteja insirido na senha e que a senha siga a regra que estabelecemos anteriormente
    @classmethod
    def validate_user(cls, v: dict[str, Any]) -> dict[str, Any]:
        if "name" not in v or "password" not in v:
            raise ValueError("Name and password are required")
        if v["name"].casefold() in v["password"].casefold():
            raise ValueError("Password cannot contain name")
        if not VALID_PASSWORD_REGEX.match(v["password"]):
            raise ValueError(
                "Password is invalid, must contain 8 characters, 1 uppercase, 1 lowercase, 1 number"
            )
        v["password"] = hashlib.sha256(v["password"].encode()).hexdigest()
        return v

Aqui estão exemplos de dados que devem ser seguidos como exemplos corretos e exemplos errados

In [ ]:
def validate(data: dict[str, Any]) -> None:
    try:
        user = User.model_validate(data)
        print(user)
    except ValidationError as e:
        print("User is invalid:")
        print(e)


def main() -> None: #exemplos de dados que podem ser gerados
    test_data = dict(
        good_data={ #dados bons
            "name": "Arjan",
            "email": "example@arjancodes.com",
            "password": "Password123",
            "role": "Admin",
        },
        bad_role={ #ausencia de role
            "name": "Arjan",
            "email": "example@arjancodes.com",
            "password": "Password123",
            "role": "Programmer",
        },
        bad_data={ #dados ruins (email e senha)
            "name": "Arjan",
            "email": "bad email",
            "password": "bad password",
        },
        bad_name={ #nome fora do padrão
            "name": "Arjan<-_->",
            "email": "example@arjancodes.com",
            "password": "Password123",
        },
        duplicate={ #dados duplicados
            "name": "Arjan",
            "email": "example@arjancodes.com",
            "password": "Arjan123",
        },
        missing_data={ #dados faltantes
            "email": "",
            "password": "",
        },
    )

    for example_name, data in test_data.items():
        print(example_name)
        validate(data)
        print()

Por fim podemos ver a validação dos codigos que fizemos até agora, mostrando qual é o acerto e erro em cada exemplo: good_data, bad_role, duplicate, missing_data, bad_name e bad_data.

In [ ]:
if __name__ == "__main__":
    main()

good_data
name='Arjan' email='example@arjancodes.com' password=SecretStr('**********') role=<Role.Admin: 4>

bad_role
User is invalid:
1 validation error for User
role
  Value error, Role is invalid, please use one of the following: Author, Editor, Admin, SuperAdmin [type=value_error, input_value='Programmer', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error

bad_data
User is invalid:
1 validation error for User
  Value error, Password is invalid, must contain 8 characters, 1 uppercase, 1 lowercase, 1 number [type=value_error, input_value={'name': 'Arjan', 'email'...ssword': 'bad password'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error

bad_name
User is invalid:
1 validation error for User
name
  Value error, Name is invalid, must contain only letters and be at least 2 characters long [type=value_error, input_value='Arjan<-_->', input_type=str]
    For further information visit https://

# **Terceiro Passo**

Agora faremos a serialização personalizada, ou seja transformaremos um objetos em outra estrutura de dados, como por exemplo JSON, utilizaremos muito quando incluirmos a biblioteca FastAPI.

In [ ]:
import enum
import hashlib
import re
from typing import Any, Self
from pydantic import (BaseModel,EmailStr,Field,field_serializer,field_validator,model_serializer,model_validator,SecretStr,)

In [ ]:
#define regras para a criação de senha e nome do ususário
VALID_PASSWORD_REGEX = re.compile(r"^(?=.*[a-z])(?=.*[A-Z])(?=.*\d).{8,}$") # a senha deve possuir no mínimo 8 caracteres, sendo pelo menos 1 letra maiúscula, 1 letra minúscula e 1 número.
VALID_NAME_REGEX = re.compile(r"^[a-zA-Z]{2,}$")# o nome deve possuir no mínimo 2 caracteres, sendo maiusculas e também minusculas.

In [ ]:
class Role(enum.IntFlag): #Criação das roles agora com valores int
    User = 0
    Author = 1
    Editor = 2
    Admin = 4
    SuperAdmin = 8


class User(BaseModel):
    name: str = Field(examples=["Example"])
    email: EmailStr = Field(
        examples=["user@arjancodes.com"],
        description="The email address of the user",
        frozen=True,
    )
    password: SecretStr = Field(
        examples=["Password123"], description="The password of the user", exclude=True #Quando fizermos a serialização desse objeto, essa senha não será incluida
    )
    role: Role = Field(
        description="The role of the user",
        examples=[1, 2, 4, 8],# exemplo das roles com int
        default=0, #Como estamos dando valor à role, agora o seu defult será 0 em vez de none
        validate_default=True,
    )

    @field_validator("name") #validaremos se o nome seque a regra criada anteiormente (VALID_NAME_REGEX)
    def validate_name(cls, v: str) -> str:
        if not VALID_NAME_REGEX.match(v):
            raise ValueError(
                "Name is invalid, must contain only letters and be at least 2 characters long"
            )
        return v

    @field_validator("role", mode="before") #validaremos se o valor inserio na role está entre o int selecionados
    @classmethod
    def validate_role(cls, v: int | str | Role) -> Role:
        op = {int: lambda x: Role(x), str: lambda x: Role[x], Role: lambda x: x}
        try:
            return op[type(v)](v)
        except (KeyError, ValueError):
            raise ValueError(
                f"Role is invalid, please use one of the following: {', '.join([x.name for x in Role])}"
            )

    @model_validator(mode="before") #validaremos se existe um nome e se a senha segue a regra criada anteriormente (VALID_PASSWORD_REGEX)
    @classmethod
    def validate_user_pre(cls, v: dict[str, Any]) -> dict[str, Any]:
        if "name" not in v or "password" not in v:
            raise ValueError("Name and password are required")
        if v["name"].casefold() in v["password"].casefold():
            raise ValueError("Password cannot contain name")
        if not VALID_PASSWORD_REGEX.match(v["password"]):
            raise ValueError(
                "Password is invalid, must contain 8 characters, 1 uppercase, 1 lowercase, 1 number"
            )
        v["password"] = hashlib.sha256(v["password"].encode()).hexdigest()
        return v

    @model_validator(mode="after") #valida que só exista um admin
    def validate_user_post(self, v: Any) -> Self:
        if self.role == Role.Admin and self.name != "Arjan":
            raise ValueError("Only Arjan can be an admin")
        return self

    @field_serializer("role", when_used="json")#Aqui temos um exemplo, quando fizermos a serialização do objeto, garantimos que o nome da role seja retornado para que possamos incluir em dados no formato JSON
    @classmethod
    def serialize_role(cls, v) -> str:
        return v.name

    @model_serializer(mode="wrap", when_used="json") #serelização da função usuário para json
    def serialize_user(self, serializer, info) -> dict[str, Any]:
        if not info.include and not info.exclude:
            return {"name": self.name, "role": self.role.name}
        return serializer(self)

In [ ]:
def main() -> None:
    data = {
        "name": "Arjan",
        "email": "example@arjancodes.com",
        "password": "Password123",
        "role": "Admin",
    }
    user = User.model_validate(data) #validação e criação do Usuário user
    if user:
        print(
            "The serializer that returns a dict:", #serialização dos dados para um dicionário
            user.model_dump(),
            sep="\n",
            end="\n\n",
        )
        print(
            "The serializer that returns a JSON string:", #serialização dos dados para um JSON
            user.model_dump(mode="json"),
            sep="\n",
            end="\n\n",
        )
        print(
            "The serializer that returns a json string, excluding the role:", #serialização dos dados para um JSON, excluindo a role
            user.model_dump(exclude=["role"], mode="json"),
            sep="\n",
            end="\n\n",
        )
        print("The serializer that encodes all values to a dict:", dict(user), sep="\n") #serialização dos dados para um dicionário junto com a role

In [ ]:
if __name__ == "__main__": #executando-a já podemos ver como a serialização ficará
    main()

The serializer that returns a dict:
{'name': 'Arjan', 'email': 'example@arjancodes.com', 'role': <Role.Admin: 4>}

The serializer that returns a JSON string:
{'name': 'Arjan', 'role': 'Admin'}

The serializer that returns a json string, excluding the role:
{'name': 'Arjan', 'email': 'example@arjancodes.com'}

The serializer that encodes all values to a dict:
{'name': 'Arjan', 'email': 'example@arjancodes.com', 'password': SecretStr('**********'), 'role': <Role.Admin: 4>}


# **Quarto Passo**

Utilização da FastAPI, a sua integração junto ao Pydantic, permitindo o trabalho com API's

In [ ]:
%pip install FastAPI

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.6 MB/s eta 0:00:00


In [ ]:
from datetime import datetime
from typing import Optional
from uuid import uuid4

from fastapi import FastAPI
from fastapi.responses import JSONResponse
from fastapi.testclient import TestClient
from pydantic import BaseModel, EmailStr, Field, field_serializer, UUID4

app = FastAPI()

In [ ]:
class User(BaseModel):
    model_config = {
        "extra": "forbid", #Proibe que atributos extras sejam adicionados
    }
    __users__ = [] #O restanta continua muito parecido com os anteriores, porem agora aplicaremos isso a uma API
    name: str = Field(..., description="Name of the user") #Além disso adicionamos mais variáveis como: friends, blocked, signup_ts e id
    email: EmailStr = Field(..., description="Email address of the user")
    friends: list[UUID4] = Field(
        default_factory=list, max_items=500, description="List of friends"
    )
    blocked: list[UUID4] = Field(
        default_factory=list, max_items=500, description="List of blocked users"
    )
    signup_ts: Optional[datetime] = Field(
        default_factory=datetime.now, description="Signup timestamp", kw_only=True
    )
    id: UUID4 = Field(
        default_factory=uuid4, description="Unique identifier", kw_only=True
    )

    @field_serializer("id", when_used="json")#Irá serializar o 'id' para json
    def serialize_id(self, id: UUID4) -> str:
        return str(id)

Aqui irei colocar em prática a biblioteca FastAPI.

In [ ]:
@app.get("/users", response_model=list[User]) #endpoint responsável por retornar a lista de usuarios criados
async def get_users() -> list[User]:
    return list(User.__users__)


@app.post("/users", response_model=User)  #endpoint responsável por criar um novo usuário
async def create_user(user: User):
    User.__users__.append(user)
    return user


@app.get("/users/{user_id}", response_model=User) #endpoint responsável por retornar um usuário específico, por meio do seu id
async def get_user(user_id: UUID4) -> User | JSONResponse:
    try:
        return next((user for user in User.__users__ if user.id == user_id))
    except StopIteration:
        return JSONResponse(status_code=404, content={"message": "User not found"}) #caso não exista ira printar um erro

Agora iremos fazer a verificação dos dados, criando exemplos para serem verificados, gerando mensagens que expliquem os erros caso existam.

In [ ]:
def main() -> None:
    with TestClient(app) as client:
        for i in range(5): #crição de 5 usuários
            response = client.post(
                "/users",
                json={"name": f"User {i}", "email": f"example{i}@arjancodes.com"},
            )
            assert response.status_code == 200
            assert response.json()["name"] == f"User {i}", (#verifica o nome do paciente
                "The name of the user should be User {i}"
            )
            assert response.json()["id"], "The user should have an id" #verifica se o paciente tem um id

            user = User.model_validate(response.json()) #converte de json para objeto 'user' para validação
            assert str(user.id) == response.json()["id"], "The id should be the same" #verifica se o id do paciente é o mesmo que está no json
            assert user.signup_ts, "The signup timestamp should be set" #verifica se a data da consulta foi definida
            assert user.friends == [], "The friends list should be empty" #verifica se a lista de amigos está vazia
            assert user.blocked == [], "The blocked list should be empty" #verifica se a lista de bloqueados está vazia

        response = client.get("/users")  #obtem a lista de usuários criada anteriormente
        assert response.status_code == 200, "Response code should be 200"
        assert len(response.json()) == 5, "There should be 5 users" #verifica se foram criados 5 usuários

        response = client.post( #outra verificação, agora com somente 1 usuário, porém os passos são identicos
            "/users", json={"name": "User 5", "email": "example5@arjancodes.com"}
        )
        assert response.status_code == 200
        assert response.json()["name"] == "User 5", (
            "The name of the user should be User 5"
        )
        assert response.json()["id"], "The user should have an id"

        user = User.model_validate(response.json())
        assert str(user.id) == response.json()["id"], "The id should be the same"
        assert user.signup_ts, "The signup timestamp should be set"
        assert user.friends == [], "The friends list should be empty"
        assert user.blocked == [], "The blocked list should be empty"

        response = client.get(f"/users/{response.json()['id']}") #verifica a crição de um usuário, no caso o ultimo criado
        assert response.status_code == 200
        assert response.json()["name"] == "User 5", (
            "This should be the newly created user"
        )

        response = client.get(f"/users/{uuid4()}") #verifica a criação de um usuário que no caso não existe
        assert response.status_code == 404
        assert response.json()["message"] == "User not found", (
            "We technically should not find this user"
        )

        response = client.post("/users", json={"name": "User 6", "email": "wrong"}) #verificação do email
        assert response.status_code == 422, "The email address is should be invalid"

In [ ]:
if __name__ == "__main__":
    main()